---
## 🔧 Step 0: Setup

In [ ]:
%run ../00_setup

In [ ]:
# Table from previous workshops
table_name = f"{catalog}.{schema}.customers_silver"
print(f"📊 Working with table: {table_name}")

# Check if table exists
display(spark.sql(f"DESCRIBE TABLE {table_name}"))

---
## 👀 Step 1: Review Current Permissions

### Task 1.1: Check who has access to the table

**Hint:**
```sql
SHOW GRANTS ON TABLE catalog.schema.table_name
```

In [ ]:
# TODO: Display current permissions on the table
# display(spark.sql(f"SHOW GRANTS ON TABLE {table_name}"))

### Task 1.2: Check schema permissions

In [ ]:
# TODO: Display schema permissions
# display(spark.sql(f"SHOW GRANTS ON SCHEMA {catalog}.{schema}"))

---
## 🔑 Step 2: Permission Management (GRANT/REVOKE)

### Business Scenario

The company has 3 roles:
1. **Analysts** (`analysts`) - read-only access to aggregated data
2. **Data Scientists** (`data_scientists`) - read access to detailed data
3. **Data Engineers** (`data_engineers`) - full access

### Task 2.1: Grant SELECT permission for analysts

**Hint:**
```sql
GRANT SELECT ON TABLE catalog.schema.table TO `group_name`
```

In [ ]:
# First check available groups (if you have permissions)
# display(spark.sql("SHOW GROUPS"))

In [ ]:
# TODO: Grant SELECT permission for 'account users' group (or another available)
user_or_group = "account users"  # Change to an actual group in your environment

# spark.sql(f"GRANT SELECT ON TABLE {table_name} TO `{user_or_group}`")
# print(f"✅ Granted SELECT to: {user_or_group}")

### Task 2.2: Revoke permission (optional)

**Hint:**
```sql
REVOKE SELECT ON TABLE catalog.schema.table FROM `group_name`
```

In [ ]:
# TODO: Revoke permission (optional)
# spark.sql(f"REVOKE SELECT ON TABLE {table_name} FROM `{user_or_group}`")

---
## 🎭 Step 3: Data Masking (Dynamic Views)

### Problem

Analysts need access to the customer table, but CANNOT see:
- Full email address
- Phone number

### Solution: Dynamic View

A view that masks data depending on WHO is querying.

### Task 3.1: Create a view with masking

**Requirements:**
- Members of the `admins` group see full data
- Others see masked email: `***@***.com`
- Others see masked phone: `***-***-****`

**Hint:**
```sql
CREATE OR REPLACE VIEW view_name AS
SELECT 
    column1,
    CASE 
        WHEN is_account_group_member('admins') THEN sensitive_column
        ELSE '***masked***'
    END AS sensitive_column
FROM source_table
```

In [ ]:
view_name = f"{catalog}.{schema}.customers_masked"

# TODO: Create view with masking
sql_create_view = f"""
CREATE OR REPLACE VIEW {view_name} AS
SELECT 
    CustomerID,
    FirstName,
    LastName,
    FullName,
    CompanyName,
    -- TODO: Mask EmailAddress
    -- CASE 
    --     WHEN is_account_group_member('admins') THEN EmailAddress
    --     ELSE CONCAT(SUBSTRING(EmailAddress, 1, 2), '***@***.com')
    -- END AS EmailAddress,
    EmailAddress,  -- <- Replace with the CASE above
    -- TODO: Mask Phone
    Phone,
    City,
    StateProvince,
    CountryRegion
FROM {table_name}
"""

# spark.sql(sql_create_view)
# print(f"✅ Created view: {view_name}")

### Task 3.2: Test the masking

Check how the data looks for you (as the current user).

In [ ]:
# Check who is currently querying
current_user = spark.sql("SELECT current_user() as user").collect()[0]["user"]
print(f"👤 Current user: {current_user}")

# Check if you're in the admins group
is_admin = spark.sql("SELECT is_account_group_member('admins') as is_admin").collect()[0]["is_admin"]
print(f"🔐 Is admin? {is_admin}")

In [ ]:
# Display data from view
# display(spark.table(view_name).limit(10))

---
## 🔍 Step 4: Row-Level Security (RLS)

### Scenario

Regional managers should see ONLY customers from their region.

### Task 4.1: Create a Row Filter

**Hint:**
```sql
CREATE FUNCTION catalog.schema.filter_by_region(region STRING)
RETURNS BOOLEAN
RETURN (
    is_account_group_member('admins') 
    OR region = 'Washington'  -- or dynamically based on user
);

ALTER TABLE table_name SET ROW FILTER catalog.schema.filter_by_region ON (StateProvince);
```

In [ ]:
# TODO: Create filter function (optional - requires permissions)
# filter_function = f"{catalog}.{schema}.filter_by_region"

# sql_create_filter = f"""
# CREATE OR REPLACE FUNCTION {filter_function}(region STRING)
# RETURNS BOOLEAN
# RETURN (
#     is_account_group_member('admins') 
#     OR region = 'Washington'
# )
# """

# spark.sql(sql_create_filter)
# print(f"✅ Created function: {filter_function}")

In [ ]:
# TODO: Apply Row Filter to table
# spark.sql(f"ALTER TABLE {table_name} SET ROW FILTER {filter_function} ON (StateProvince)")

---
## 📊 Step 5: Audit - System Tables

### Task 5.1: Check access logs

Unity Catalog automatically logs every access. System Tables are available in `system.access.audit`.

**Note:** Requires appropriate permissions to System Tables.

In [ ]:
# Check audit logs (if you have access)
try:
    audit_query = f"""
    SELECT 
        event_time,
        user_identity.email as user_email,
        action_name,
        request_params.full_name_arg as table_accessed
    FROM system.access.audit
    WHERE action_name IN ('getTable', 'commandSubmit')
    AND request_params.full_name_arg LIKE '%customers%'
    ORDER BY event_time DESC
    LIMIT 20
    """
    display(spark.sql(audit_query))
except Exception as e:
    print(f"⚠️ No access to System Tables: {e}")

### Task 5.2: Check Lineage (data origin)

In the Unity Catalog UI:
1. Go to your table
2. Click the **Lineage** tab
3. See where the data comes from and where it's used

In [ ]:
# Check table dependencies (programmatically)
# display(spark.sql(f"DESCRIBE EXTENDED {table_name}"))

---
## 🧹 Cleanup (optional)

In [ ]:
# Remove created objects (if you want to clean up)
# spark.sql(f"DROP VIEW IF EXISTS {view_name}")
# spark.sql(f"DROP FUNCTION IF EXISTS {filter_function}")
# spark.sql(f"ALTER TABLE {table_name} DROP ROW FILTER")

---
---

# 📋 SOLUTION

⚠️ **Don't look here until you've tried it yourself!** ⚠️

In [ ]:
# ============================================================
# 📋 FULL SOLUTION - Workshop 4: Governance & Security
# ============================================================

table_name = f"{catalog}.{schema}.customers_silver"
view_name = f"{catalog}.{schema}.customers_masked"
user_or_group = "account users"

# --- Step 1: Review permissions ---
print("🔍 CURRENT PERMISSIONS ON TABLE:")
display(spark.sql(f"SHOW GRANTS ON TABLE {table_name}"))

# --- Step 2: Grant permissions ---
print(f"\n🔑 GRANTING SELECT TO: {user_or_group}")
spark.sql(f"GRANT SELECT ON TABLE {table_name} TO `{user_or_group}`")
print("✅ Permission granted!")

# --- Step 3: Dynamic View with masking ---
print(f"\n🎭 CREATING VIEW WITH MASKING: {view_name}")

sql_create_view = f"""
CREATE OR REPLACE VIEW {view_name} AS
SELECT 
    CustomerID,
    FirstName,
    LastName,
    FullName,
    CompanyName,
    CASE 
        WHEN is_account_group_member('admins') THEN EmailAddress
        ELSE CONCAT(SUBSTRING(EmailAddress, 1, 2), '***@***.com')
    END AS EmailAddress,
    CASE 
        WHEN is_account_group_member('admins') THEN Phone
        ELSE '***-***-****'
    END AS Phone,
    City,
    StateProvince,
    CountryRegion
FROM {table_name}
"""

spark.sql(sql_create_view)
print("✅ View created!")

# --- Verification ---
print("\n📊 DATA FROM VIEW (as current user):")
current_user = spark.sql("SELECT current_user()").collect()[0][0]
print(f"👤 User: {current_user}")
display(spark.table(view_name).limit(5))

# --- Grant access to view ---
print(f"\n🔑 GRANTING SELECT ON VIEW TO: {user_or_group}")
spark.sql(f"GRANT SELECT ON VIEW {view_name} TO `{user_or_group}`")
print("✅ Now analysts can query the masked view!")

print("\n" + "="*60)
print("✅ WORKSHOP 4 COMPLETED!")
print("="*60)